In [ ]:
!pip install face-recognition

     |████████████████████████████████| 100.2MB 42kB/s 
  Created wheel for face-recognition-models: filename=face_recognition_models-0.3.0-py2.py3-none-any.whl size=100566173 sha256=cbd1773a3277f38cd62781913f5567c3a560f21326c7c044bded4d0a75b05849
  Stored in directory: /root/.cache/pip/wheels/d2/99/18/59c6c8f01e39810415c0e63f5bede7d83dfb0ffc039865465f
Successfully built face-recognition-models


Imports required for face recognition

In [ ]:
import cv2
import numpy as np
import face_recognition
from google.colab.patches import cv2_imshow
from IPython.display import clear_output
from IPython.display import Image

getEncoding fuction contains:-


1.   loading an image
2.   converting image from BGR to RGB
3.   create encodings of image



In [ ]:
def getEncodings(name, path):
    refDict = {}
    refIMG = face_recognition.load_image_file(path)
    refIMG = cv2.cvtColor(refIMG, cv2.COLOR_BGR2RGB)
    faceENCODE = face_recognition.face_encodings(refIMG)[0]
    refDict[name] = faceENCODE
    print("refncode")
    return refDict

process function contains
1.   to save video
2.   comparing encodings of known faces
3.   draw rectangle around face



In [ ]:
def process(cap,name):
    global referenceIMG, saveVid
    if saveVid:
 
        ret,frame = cap.read()
        frame = cv2.resize(frame, (0, 0), fx=0.5, fy=0.5)
 
        size = (frame.shape[1],frame.shape[0])
 
 
        Vidresult = cv2.VideoWriter('/content/sample_data/processed.avi', cv2.VideoWriter_fourcc(*'MJPG'),
                                 10, size)
    flag = True
    while cap.isOpened():
        ret, frame = cap.read()
        if flag:
 
            frame = cv2.resize(frame, (0, 0), fx=0.5, fy=0.5)
            frameFaceLOC = face_recognition.face_locations(frame)
            frameFaceENCODE = face_recognition.face_encodings(frame)
 
            for i in range(len(frameFaceENCODE)):
                result = face_recognition.compare_faces([referenceIMG[name]], frameFaceENCODE[i])
                dist = face_recognition.face_distance([referenceIMG[name]], frameFaceENCODE[i])
                print(result)
                if result and dist[0] < 0.7:
                    print(dist)
                    cv2.rectangle(frame, (frameFaceLOC[i][3], frameFaceLOC[i][0]),
                                  (frameFaceLOC[i][1], frameFaceLOC[i][2]),
                                  (0, 255, 0), 2)
                    cv2.putText(frame, name, (frameFaceLOC[i][1] - 40, frameFaceLOC[i][2] + 20),
                                cv2.FONT_HERSHEY_COMPLEX,
                                1, (0, 0, 255), 1)
                    break
            if saveVid:
                Vidresult.write(frame)
            clear_output(wait=True)
            cv2_imshow(frame)
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break
        flag = not flag
    if saveVid:
        Vidresult.release()
    cap.release()
    cv2.destroyAllWindows()

Main

In [ ]:
name = "elon musk"
pathToReference = "/content/sample_data/elon1.jpg"
referenceIMG = getEncodings(name, pathToReference)
 
webcam = int(input("Press 1 for webcam\nPress 2 for video\n"))
saveVid = int(input("Do you want to save video? \nIf YES Press 1\nIf NO Press 0\n "))
 
if webcam == 2:
    pathToVideo = "/content/sample_data/elonvideo.mp4"
    cap = cv2.VideoCapture(pathToVideo)
 
elif webcam == 1:
    cap = cv2.VideoCapture(0)
else:
    print("INVALID INPUT")
 
if not cap.isOpened():
    print('Error opening video  file')
else:
    process(cap,name)